# Description

**TODO**

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
from pathlib import Path

import pandas as pd

import conf

# Settings

In [3]:
# if True, then it doesn't check if result files already exist and runs everything again
FORCE_RUN = True

In [4]:
PREDICTION_METHOD = "Module-based"

In [5]:
OUTPUT_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"]
display(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses')

In [6]:
# OUTPUT_DATA_DIR = Path(OUTPUT_DIR, "data")
# display(OUTPUT_DATA_DIR)
# OUTPUT_DATA_DIR.mkdir(parents=True, exist_ok=True)

In [7]:
INPUT_DATA_DIR = Path(
    OUTPUT_DIR,
    "data",
    "proj",
)
display(INPUT_DATA_DIR)
INPUT_DATA_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/proj')

In [8]:
OUTPUT_PREDICTIONS_DIR = Path(OUTPUT_DIR, "predictions")
display(OUTPUT_PREDICTIONS_DIR)
OUTPUT_PREDICTIONS_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions')

# Load PharmacotherapyDB gold standard

In [9]:
gold_standard = pd.read_pickle(
    Path(OUTPUT_DIR, "gold_standard.pkl"),
)

In [10]:
display(gold_standard.shape)

(998, 3)

In [11]:
display(gold_standard.head())

,trait,drug,true_class
0,DOID:10652,DB00843,1
1,DOID:10652,DB00674,1
2,DOID:10652,DB01043,1
3,DOID:10652,DB00989,1
4,DOID:10652,DB00810,0


In [12]:
doids_in_gold_standard = set(gold_standard["trait"])

# Load LINCS

## Projected data

In [13]:
input_file = Path(INPUT_DATA_DIR, "lincs-projection.pkl").resolve()

display(input_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/proj/lincs-projection.pkl')

In [14]:
lincs_projection = pd.read_pickle(input_file)

In [15]:
display(lincs_projection.shape)

(987, 1170)

In [16]:
display(lincs_projection.head())

perturbagen,DB00014,DB00091,DB00121,DB00130,DB00131,DB00132,DB00136,DB00140,DB00146,DB00150,...,DB08995,DB09002,DB09004,DB09009,DB09010,DB09015,DB09019,DB09020,DB09022,DB09023
LV1,0.036115,-0.358610,0.091067,0.008923,0.046469,0.136747,0.036151,0.034024,-0.027445,0.060644,...,0.069930,0.021537,0.020511,-0.006250,-0.002660,-0.017210,-0.004029,-0.145490,0.102504,0.057621
LV2,0.012281,0.009738,0.004650,-0.006554,0.021758,0.010425,0.000794,-0.000603,0.011534,-0.004425,...,0.000344,0.001123,-0.012675,-0.004849,0.000375,0.002623,-0.003252,-0.066209,0.025378,0.012453
LV3,-0.005533,-0.059174,0.013454,0.008906,-0.004939,0.033663,-0.010045,0.003824,0.009814,-0.007492,...,-0.021941,-0.002417,0.000155,0.002941,-0.015100,-0.016372,0.012249,0.009216,0.018254,-0.003516
LV4,-0.004151,-0.032884,0.005934,-0.003991,-0.028524,0.002709,0.007001,0.024704,0.005058,-0.004345,...,0.013806,0.003197,0.013099,0.002643,-0.008836,0.023852,0.028957,0.014681,0.000826,-0.009739
LV5,-0.015156,-0.005276,0.025747,0.000346,-0.013932,-0.010911,0.041403,-0.018285,0.003507,-0.007621,...,-0.011577,-0.015956,-0.027614,-0.001707,-0.000542,-0.016918,0.001959,0.024147,0.013004,-0.014166


# Load S-PrediXcan

In [17]:
from entity import Trait

In [18]:
phenomexcan_input_file_list = [
    f
    for f in INPUT_DATA_DIR.glob("*.pkl")
    if f.name.startswith("spredixcan-")
    #     if f.name.startswith(("smultixcan-", "spredixcan-"))
]

In [19]:
display(len(phenomexcan_input_file_list))

1

In [21]:
pd.read_pickle(phenomexcan_input_file_list[0]).head()

,20096_1-Size_of_red_wine_glass_drunk_small_125ml,2345-Ever_had_bowel_cancer_screening,N49-Diagnoses_main_ICD10_N49_Inflammatory_disorders_of_male_genital_organs_not_elsewhere_classified,100011_raw-Iron,5221-Index_of_best_refractometry_result_right,20003_1141150624-Treatmentmedication_code_zomig_25mg_tablet,S69-Diagnoses_main_ICD10_S69_Other_and_unspecified_injuries_of_wrist_and_hand,20024_1136-Job_code_deduced_Information_and_communication_technology_managers,20002_1385-Noncancer_illness_code_selfreported_allergy_or_anaphylactic_reaction_to_food,G6_SLEEPAPNO-Sleep_apnoea,...,Astle_et_al_2016_Sum_basophil_neutrophil_counts,RA_OKADA_TRANS_ETHNIC,pgc.scz2,PGC_ADHD_EUR_2017,MAGIC_FastingGlucose,Astle_et_al_2016_Red_blood_cell_count,SSGAC_Depressive_Symptoms,BCAC_ER_positive_BreastCancer_EUR,IBD.EUR.Inflammatory_Bowel_Disease,Astle_et_al_2016_High_light_scatter_reticulocyte_count
LV1,0.005032,-0.017742,-0.000063,-0.027630,-0.035386,0.011002,0.010724,0.010446,-0.018710,0.010722,...,-0.077884,-0.059427,0.014250,0.011559,0.010034,-0.007861,0.050615,-0.024714,-0.016299,0.012266
LV2,0.022757,-0.002494,-0.022456,-0.006039,0.014805,0.029324,-0.020111,-0.029098,-0.028098,-0.011183,...,0.022188,-0.004523,0.044796,-0.003497,0.036967,0.018417,-0.002459,-0.019084,0.023447,-0.014293
LV3,-0.014583,-0.009334,-0.008777,-0.016107,-0.026012,-0.006325,-0.019086,-0.027593,0.009968,-0.026543,...,0.041333,-0.015025,-0.050110,0.034439,-0.023318,-0.030491,-0.003833,-0.017156,0.013962,-0.053326
LV4,-0.007432,-0.029053,-0.031635,-0.016797,0.015012,0.034683,-0.013831,0.037747,-0.040881,0.006171,...,0.029442,-0.005255,0.006472,0.003765,-0.023333,-0.057144,0.018355,0.048978,0.032675,0.041624
LV5,-0.035077,-0.038190,-0.008507,0.000203,-0.010666,-0.002614,0.013233,-0.005662,-0.022220,0.000116,...,0.036306,-0.058390,-0.036769,0.014124,-0.010000,-0.008742,0.025883,-0.023848,-0.034297,-0.003898


# Predict drug-disease associations

In [22]:
from drug_disease import (
    #     predict_dotprod,
    predict_dotprod_neg,
    #     predict_pearson,
    #     predict_pearson_neg,
    #     predict_spearman,
    #     predict_spearman_neg,
)

In [23]:
methods_to_run = [predict_dotprod_neg]

In [24]:
for phenomexcan_input_file in phenomexcan_input_file_list:
    print(phenomexcan_input_file.name)

    # read phenomexcan data
    phenomexcan_projection = pd.read_pickle(phenomexcan_input_file)
    print(f"  shape: {phenomexcan_projection.shape}")

    for prediction_method in methods_to_run:
        for ntc in (None, 5, 10, 25, 50):
            prediction_method(
                lincs_projection,
                phenomexcan_input_file,
                phenomexcan_projection,
                OUTPUT_PREDICTIONS_DIR,
                PREDICTION_METHOD,
                doids_in_gold_standard,
                FORCE_RUN,
                n_top_conditions=ntc,
                use_abs=True,
            )

            print("\n")

spredixcan-most_signif-projection.pkl
  shape: (987, 4091)
  predicting all_genes...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


,score
count,410670.000000
mean,0.004373
std,0.028793
min,-1.032507
25%,-0.005697
50%,0.003642
75%,0.012913
max,1.266442


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/spredixcan-most_signif-projection-all_genes-prediction_scores.h5


  predicting top_5_genes...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


,score
count,410670.000000
mean,0.000938
std,0.006307
min,-0.264044
25%,-0.001053
50%,0.000784
75%,0.002655
max,0.297101


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/spredixcan-most_signif-projection-top_5_genes-prediction_scores.h5


  predicting top_10_genes...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


...

    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/spredixcan-most_signif-projection-top_10_genes-prediction_scores.h5


  predicting top_25_genes...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


,score
count,410670.000000
mean,0.001784
std,0.011800
min,-0.331659
25%,-0.002215
50%,0.001507
75%,0.005258
max,0.527264


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/spredixcan-most_signif-projection-top_25_genes-prediction_scores.h5


  predicting top_50_genes...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


,score
count,410670.000000
mean,0.002357
std,0.015196
min,-0.569921
25%,-0.002917
50%,0.001962
75%,0.006876
max,0.642653


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/spredixcan-most_signif-projection-top_50_genes-prediction_scores.h5


